(numpy)=
# Numpy

Широко используемая библиотека для вычислений с многомерными массивами. API большей частью вдохновлен MATLAB (великая и ужасная среда, язык и IDE для матричных вычислений), а теперь сам является примером для подражания API различных вычислительных пакетов.
Более последовательный гайд стоит посмотреть на [сайте библиотеки](https://numpy.org/devdocs/user)


## Массивы

In [1]:
import numpy as np

a = np.array([1, 2, 3]) # создадим вектор
print(f"{a = }")

b = np.zeros((2, 2))
print(f"{b = }")

c = np.eye(3)
print(f"{c = }")

q = np.random.random((1, 100))
print(f"{q = }")

a = array([1, 2, 3])
b = array([[0., 0.],
       [0., 0.]])
c = array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])
q = array([[0.57082139, 0.36993029, 0.45401214, 0.33005512, 0.95674083,
        0.56040096, 0.76824237, 0.38268489, 0.73166959, 0.87361145,
        0.12698558, 0.92720012, 0.77417621, 0.78957571, 0.44840018,
        0.00716917, 0.64616837, 0.19758688, 0.63979546, 0.59636372,
        0.61757197, 0.22045616, 0.23542215, 0.90696162, 0.79538996,
        0.47896193, 0.88868939, 0.41718578, 0.00407227, 0.71176224,
        0.9534245 , 0.46106395, 0.21498149, 0.00511336, 0.90288084,
        0.69911752, 0.46673178, 0.75085726, 0.9203732 , 0.88056869,
        0.29311809, 0.4974451 , 0.06512541, 0.26783214, 0.5687371 ,
        0.8912709 , 0.40351702, 0.52259953, 0.81962311, 0.7531149 ,
        0.88216758, 0.16963851, 0.01449336, 0.83310114, 0.87939821,
        0.77549904, 0.23409649, 0.85438279, 0.83709866, 0.95591167,
        0.94138284, 0.91189771, 0.96388879, 0.384869

## Math ops
Для удобства использования np.ndarray определены арифметические операторы, так чтобы соответствовать ожиданиям:

In [2]:
a = np.array([1, 2, 3])
b = np.array([-1, 3, 4])

diff = a - b
print(f"{diff = }")

mult = a * b
print(f"{mult = }")

scalar_mult = a @ b
print(f"{scalar_mult = }")

diff = array([ 2, -1, -1])
mult = array([-1,  6, 12])
scalar_mult = 17


## Indexing, slicing and sugar

Numpy поддерживает кажется все разумные варианты индексации:

In [3]:
a = np.arange(16).reshape(4, 4)
print(f"{a = }")

# просто по индексам
print(f"a_{0,1}: {a[0, 1] = }, {a[0][1] = }")

# по слайсам
print("a_{1,1..3}", a[0, 1:3])
print(f"a_{2}: {a[2] = }, {a[2, :] = }, {a[2, ...] = }")

# по маске
mask = (a % 3 == 0)
print(f"{mask = }")
print(f"{a[mask] = }")

first_rows = np.array([True, True, False, False])
print(f"{a[first_rows] = }")

a = array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])
a_(0, 1): a[0, 1] = 1, a[0][1] = 1
a_{1,1..3} [1 2]
a_2: a[2] = array([ 8,  9, 10, 11]), a[2, :] = array([ 8,  9, 10, 11]), a[2, ...] = array([ 8,  9, 10, 11])
mask = array([[ True, False, False,  True],
       [False, False,  True, False],
       [False,  True, False, False],
       [ True, False, False,  True]])
a[mask] = array([ 0,  3,  6,  9, 12, 15])
a[first_rows] = array([[0, 1, 2, 3],
       [4, 5, 6, 7]])


Для работы с размерностями часто используются еще три конструкции: `None`, `...` (ellipsis, многоточие) и `:` (двоеточие).

In [4]:
a = np.arange(16).reshape(4, 4)
print(f"{a = }")

# None добавляет ось размерности 1
print(f"{a[None].shape = }")
print(f"{a[:, :, None].shape = }")

# : превращается в slice (None), берет все элементы вдоль размерности
print(f"{a[2, :] = }")
print(f"{a[2, 0:None] = }")

# ... ellipsis, превращается в необходимое число двоеточий :,:,:
print(f"{a[...] = }")

# также ... удобен когда мы не знаем настоящий шейп массива или нужно не трогать несколько подряд идущих размерностей
z = np.arange(27).reshape(3, 3, 3)
print(f"{z[0, ..., 1] = }, {z[0, :, 1] = }")

a = array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])
a[None].shape = (1, 4, 4)
a[:, :, None].shape = (4, 4, 1)
a[2, :] = array([ 8,  9, 10, 11])
a[2, 0:None] = array([ 8,  9, 10, 11])
a[...] = array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])
z[0, ..., 1] = array([1, 4, 7]), z[0, :, 1] = array([1, 4, 7])


В целом, в numpy очень здорово реализованы методы `__getitem__`/`__setitem__`.

In [5]:
a = np.array([1, 2, 3])
element = a[2]
print(f"{element = }")

a[2] = 5
print(f"{a = }")

element = 3
a = array([1, 2, 5])


Кроме того, мы можем делать индексацию по заданному условию с помощью `np.where`

In [6]:
# создадим вектор
a = np.array([2, 4, 6, 8])

selection = np.where(a < 5)
print(f"{selection = }")

# дополнительно мы можем передать два значения или вектора, при выполнении условия выбираются элементы из первого значения/вектора, при невыполнении - из второго
a2 = np.where(a < 5, 2, a * 2)
print(f"{a2 = }")

# np.where работает и с многомерными массивами
b = np.array([[8, 8, 2, 6], [0, 5, 3, 4]])
b_mult = np.where(b < 4, b, 1)
print(f"{b_mult = }")

selection = (array([0, 1]),)
a2 = array([ 2,  2, 12, 16])
b_mult = array([[1, 1, 2, 1],
       [0, 1, 3, 1]])


## Broadcasting

Что происходит, если мы хотим арифметику с массивами разных размеров?

In [7]:
a = np.array([1, 2, 3])
k = 2
broad = a * k
print(f"{broad = }")

broad = array([2, 4, 6])


С точки зрения математики, ничего интересного тут не происходит: мы подразумевали умножение всего вектора на скаляр. Однако матричные операции в numpy справляются и с менее очевидными случаями, например при сложении или вычитании вектора и скаляра:

In [8]:
a = np.array([1, 2, 3])
k = 2
broad = a - k
print(f"{broad = }")

broad = array([-1,  0,  1])


В numpy приняты следующие правила работы с массивами разного размера:

1. Размерности сравниваются справа налево
2. Два массива совместимы в размерности, если она одинаковая, либо у одного из массивов единичная.
3. Вдоль отсутствующих размерностей происходит расширение повторением (`np.repeat`).

![.](https://i.stack.imgur.com/JcKv1.png)

```{admonition} Link to the source
https://mathematica.stackexchange.com/questions/99171/how-to-implement-the-general-array-broadcasting-method-from-numpy
```

Be aware, автоматический броадкастинг легко приводит к ошибкам, так что лучше делать его самостоятельно в явной форме.


## floating point things

Отдельно стоит поговорить про числа с плавающей точкой.
Число с плавающей запятой (или число с плавающей точкой) — экспоненциальная форма представления вещественных (действительных) чисел, в которой число хранится в виде мантиссы и порядка (показателя степени). При этом число с плавающей запятой имеет фиксированную относительную точность и изменяющуюся абсолютную.
В результате одно и то же значение может выглядеть по-разному, если хранить его с разной точностью.

In [9]:
f16 = np.float16("0.1")
f32 = np.float32(f16)
f64 = np.float64(f32)
print(f"{f16 = }, {f32 = }, {f64 = }")
print(f"{f16 == f32 == f64 = }")

f16 = np.float16("0.1")
f32 = np.float32("0.1")
f64 = np.float64("0.1")
print(f"{f16 = }, {f32 = }, {f64 = }")
print(f16 == f32 == f64)

f16 = 0.1, f32 = 0.099975586, f64 = 0.0999755859375
f16 == f32 == f64 = True
f16 = 0.1, f32 = 0.1, f64 = 0.1
False


Из-за этого для сравнения массивов с типом float используют `np.allclose`.

In [10]:
print(f"{np.allclose([1e10,1e-7], [1.00001e10,1e-8]) = }")
print(f"{np.allclose([1e10,1e-8], [1.00001e10,1e-9]) = }")

np.allclose([1e10,1e-7], [1.00001e10,1e-8]) = False
np.allclose([1e10,1e-8], [1.00001e10,1e-9]) = True


## numpy & linalg fun

[Не знаю, что имеется ввиду под "Матричные трюки"]

["вычисление попарных расстояний" - вроде обычно используется scipy/sklearn?]

### Решение системы линейных уравнений

Numpy позволяет решить систему линейных уравнений.

In [11]:
a = np.array([[7, 4], [9, 8]])
b = np.array([5, 3])
solution = np.linalg.solve(a, b)
print(f"{solution = }")

solution = array([ 1.4, -1.2])


### Обращение матриц.

Numpy даёт возможность выполнить операцию обращения матриц.

In [12]:
a = np.array([[1., 2.], [3., 4.]])
inv = np.linalg.inv(a)
print(f"{inv = }")

inv = array([[-2. ,  1. ],
       [ 1.5, -0.5]])


### Собственные вектора и числа

Вычисление собственных векторов и чисел.

In [13]:
print(np.linalg.eig(np.diag((1, 2, 3))))

(array([1., 2., 3.]), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]))


## Что мы узнали

- основы работы с numpy
- индексацию в массивах
- broadcasting
- floating point things
- numpy fun